In [ ]:
!pip install mediapipe

In [9]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [10]:
data = {
    "NOSE":[],
    "LEFT_EYE_INNER":[],
    "LEFT_EYE":[],
    "LEFT_EYE_OUTER":[],
    "RIGHT_EYE_INNER":[],
    "RIGHT_EYE":[],
    "RIGHT_EYE_OUTER":[],
    "LEFT_EAR":[],
    "RIGHT_EAR":[],
    "MOUTH_LEFT":[],
    "MOUTH_RIGHT":[],
    "LEFT_SHOULDER":[],
    "RIGHT_SHOULDER":[],
    "LEFT_ELBOW":[],
    "RIGHT_ELBOW":[],
    "LEFT_WRIST":[],
    "RIGHT_WRIST":[],
    "LEFT_PINKY":[],
    "RIGHT_PINKY":[],
    "LEFT_INDEX":[],
    "RIGHT_INDEX":[],
    "LEFT_THUMB":[],
    "RIGHT_THUMB":[],
    "LEFT_HIP":[],
    "RIGHT_HIP":[],
    "LEFT_KNEE":[],
    "RIGHT_KNEE":[],
    "LEFT_ANKLE":[],
    "RIGHT_ANKLE":[],
    "LEFT_HEEL":[],
    "RIGHT_HEEL":[],
    "LEFT_FOOT_INDEX":[],
    "RIGHT_FOOT_INDEX":[]
}


In [12]:
#Video process
path = 'replace with video path'
cap = cv2.VideoCapture(path)
# Obtiene la velocidad de fotogramas (fps)
fps = cap.get(cv2.CAP_PROP_FPS)
if not cap.isOpened():
  print('el video no se detecto o fallo al abrir')
else:
  empty = []
  images = []
  frame_num = 0
  with mp_pose.Pose(
      min_detection_confidence=0.5,
      min_tracking_confidence=0.75,
      smooth_landmarks=True, 
      model_complexity = 2) as pose:
    while cap.isOpened():
      success, image = cap.read()
      frame_num += 1
      if not success:
        print("End of video.")
        break
      # To improve performance, optionally mark the image as not writeable to
      # pass by reference.
      image.flags.writeable = False
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = pose.process(image)
      #Store the data
      if  isinstance(results.pose_landmarks, type(None)):
        # Adding frame with its index
        empty.append(frame_num)
      else:
        for key,i in zip(data.keys(), range(33)):
          data[key].append([results.pose_landmarks.landmark[i].x, results.pose_landmarks.landmark[i].y, results.pose_landmarks.landmark[i].z])
      #Draw the landmarks
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)      
      mp_drawing.draw_landmarks(
          image,
          results.pose_landmarks,
          mp_pose.POSE_CONNECTIONS,
          landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
      #Save the image in a mtrix
      images.append(image)
      # Flip the image horizontally for a selfie-view display.
  #Make them numpy
  for key in data.keys():
    data[key] = np.array(data[key])      
  cap.release()

End of video.


Filtrado

In [15]:
#Media/Average movil
import numpy as np
import copy

x=0
y=1
z=2

#interpolate to fit size
def inter(original_data, new_length):
  original_indices = np.arange(len(original_data))
  new_indices = np.linspace(0, len(original_data) - 1, new_length)
  return np.interp(new_indices, original_indices, original_data)

def moving_average(data, window_size):
    media = np.convolve(data, np.ones(window_size), 'valid') / window_size
    return inter(media, len(data))

w=5
data_media = copy.deepcopy(data)
for key in data.keys():
  data_media[key][:, x] = moving_average(data[key][:, x], w)     
  data_media[key][:, y] = moving_average(data[key][:, y], w)
  data_media[key][:, z] = moving_average(data[key][:, z], w)


#Make test video
media_images = images
width=images[0].shape[1]
height=images[0].shape[0]

for keys in data_media.keys():
  for i in range(len(data_media[keys][:])):
    cv2.circle(media_images[i], (int(data_media[keys][i,x]*width), int(data_media[keys][i,y]*height)), radius=5, color=[255,0,0], thickness=-1)

"""
import moviepy.editor as mpy
# Crear un clip de ejemplo con un par de cuadros
W, H = 1280, 720
duration = 6  # duración del video en segundos
fps = 30  # cuadros por segundo
num_frames = duration * fps

# Crear un clip a partir de los cuadros
clip = mpy.ImageSequenceClip(media_images, fps=fps)

# Guardar el clip en formato mp4
clip.write_videofile('media.mp4')
"""


Moviepy - Building video media.mp4.
Moviepy - Writing video media.mp4



Moviepy - Done !
Moviepy - video ready media.mp4


In [ ]:
#Mediana movil
import numpy as np
import copy

x=0
y=1
z=2

#interpolate to fit size
def inter(original_data, new_length):
  original_indices = np.arange(len(original_data))
  new_indices = np.linspace(0, len(original_data) - 1, new_length)
  return np.interp(new_indices, original_indices, original_data)

def moving_median(data, window_size=10):   
    # Aplicar el filtro de mediana móvil
    median = np.array([np.median(data[i:i+window_size]) for i in range(len(data) - window_size + 1)])
    return inter(median, len(data))

w=10
data_median = copy.deepcopy(data)
for key in data.keys():
  data_median[key][:, x] = moving_median(data[key][:, x], w)     
  data_median[key][:, y] = moving_median(data[key][:, y], w)
  data_median[key][:, z] = moving_median(data[key][:, z], w)

"""
#Make test video
median_images = images
width=images[0].shape[1]
height=images[0].shape[0]

for keys in data_median.keys():
  for i in range(len(data_median[keys][:])):
    cv2.circle(median_images[i], (int(data_median[keys][i,x]*width), int(data_median[keys][i,y]*height)), radius=5, color=[255,0,0], thickness=-1)


import moviepy.editor as mpy
# Crear un clip de ejemplo con un par de cuadros
W, H = 1280, 720
duration = 6  # duración del video en segundos
fps = 30  # cuadros por segundo
num_frames = duration * fps

# Crear un clip a partir de los cuadros
clip = mpy.ImageSequenceClip(median_images, fps=fps)

# Guardar el clip en formato mp4
clip.write_videofile('median.mp4')
"""


Moviepy - Building video median.mp4.
Moviepy - Writing video median.mp4



Moviepy - Done !
Moviepy - video ready median.mp4


In [ ]:
#Savitzki Golay
#import numpy as np
import scipy.signal
import matplotlib.pyplot as plt

def savitzky_golay_filter(data, window_size, polynomial_order):
    return scipy.signal.savgol_filter(data, window_size, polynomial_order)

w=4
p=2


In [ ]:
#Gauss
import numpy as np
import copy
import scipy.signal

x=0
y=1
z=2

#interpolate to fit size
def inter(original_data, new_length):
  original_indices = np.arange(len(original_data))
  new_indices = np.linspace(0, len(original_data) - 1, new_length)
  return np.interp(new_indices, original_indices, original_data)

def gaussian_filter(data, window_size, std_dev):
    window = scipy.signal.windows.gaussian(window_size, std_dev)
    filter = np.convolve(data, window, 'valid') / np.sum(window)
    return inter(filter, len(data))

s=5
w=s*3
data_median = copy.deepcopy(data)
for key in data.keys():
  data_median[key][:, x] = gaussian_filter(data[key][:, x], w, s)     
  data_median[key][:, y] = gaussian_filter(data[key][:, y], w, s)
  data_median[key][:, z] = gaussian_filter(data[key][:, z], w, s)

#Make test video
median_images = images
width=images[0].shape[1]
height=images[0].shape[0]

for keys in data_median.keys():
  for i in range(len(data_median[keys][:])):
    cv2.circle(median_images[i], (int(data_median[keys][i,x]*width), int(data_median[keys][i,y]*height)), radius=5, color=[255,255,255], thickness=-1)

#Exportar video
import moviepy.editor as mpy
# Crear un clip de ejemplo con un par de cuadros
W, H = 1280, 720
duration = 6  # duración del video en segundos
fps = 30  # cuadros por segundo
num_frames = duration * fps

# Crear un clip a partir de los cuadros
clip = mpy.ImageSequenceClip(median_images, fps=fps)

# Guardar el clip en formato mp4
clip.write_videofile('median.mp4')

Moviepy - Building video median.mp4.
Moviepy - Writing video median.mp4



Moviepy - Done !
Moviepy - video ready median.mp4
